In [6]:
# Import the Python packages that will be needed.  
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import pprint as pp
import os
import numpy as np

In [2]:
# Read the beer reviews file
filename = os.path.join("beer_reviews/beer_reviews.csv")
df = pd.read_csv(filename)

In [3]:
# let's limit things to the top 250
n = 250
top_n = df.beer_name.value_counts().index[:n]
df = df[df.beer_name.isin(top_n)]

print df.head()

      brewery_id             brewery_name  review_time  review_overall  \
798         1075  Caldera Brewing Company   1212201268             4.5   
1559       11715  Destiny Brewing Company   1137124057             4.0   
1560       11715  Destiny Brewing Company   1129504403             4.0   
1563       11715  Destiny Brewing Company   1137125989             3.5   
1564       11715  Destiny Brewing Company   1130936611             3.0   

      review_aroma  review_appearance review_profilename  \
798            4.5                  4             grumpy   
1559           3.5                  4    blitheringidiot   
1560           2.5                  4        NeroFiddled   
1563           3.0                  4    blitheringidiot   
1564           3.0                  3             Gavage   

                            beer_style  review_palate  review_taste  \
798   American Double / Imperial Stout            4.0           4.5   
1559           American Pale Ale (APA)            3.

In [4]:
df.shape

(355275, 13)

In [7]:
# Create a pivot table with users as columns and the beer names as rows and the values being
# the 'review_overall'.  This pivot table is needed to determine the cosine similarity of the
# beers using the 'review_overall' values for similarity determination.
print "melting..."
df_wide = pd.pivot_table(df, values=["review_overall"],
                         index=["beer_name", "review_profilename"],
                         aggfunc=np.mean).unstack()

melting...


In [8]:
df_wide.head()

review_overall                               \
review_profilename               0110x011 02maxima 03SVTCobra 05Harley   
beer_name                                                                
#9                                    NaN      NaN        NaN      NaN   
120 Minute IPA                        NaN      NaN        NaN        4   
1554 Enlightened Black Ale            NaN      NaN        NaN      NaN   
60 Minute IPA                         NaN      NaN        NaN      NaN   
90 Minute IPA                           5      NaN        NaN        4   

                                                                         \
review_profilename         0Naught0 0beerguy0 0runkp0s 0tt0 1000Bottles   
beer_name                                                                 
#9                              NaN       NaN      NaN  NaN         NaN   
120 Minute IPA                  NaN       NaN      NaN  1.5         NaN   
1554 Enlightened Black Ale      NaN       NaN      NaN  NaN         NaN   
60 Minute IPA                   NaN       NaN      NaN  NaN         NaN   
90 Minute IPA                   NaN       NaN      NaN  NaN         NaN   

                                      ...                                      \
review_profilename         1001111.0  ...   zuker zulufactor zumicroom zwalk8   
beer_name                             ...                                       
#9                               NaN  ...     NaN        NaN       NaN    NaN   
120 Minute IPA                   NaN  ...     NaN        NaN       NaN    NaN   
1554 Enlightened Black Ale       NaN  ...     NaN        NaN       NaN    NaN   
60 Minute IPA                    NaN  ...     NaN        NaN       NaN    NaN   
90 Minute IPA                    NaN  ...     NaN        NaN       NaN    NaN   

                                                                                
review_profilename         zwoehr zymrgy zymurgy4all zymurgywhiz zythus zyzygy  
beer_name                                                                       
#9                            NaN    NaN         NaN         NaN    NaN    NaN  
120 Minute IPA                NaN    NaN         NaN         NaN    NaN    NaN  
1554 Enlightened Black Ale    NaN    NaN         NaN         NaN    NaN    NaN  
60 Minute IPA                 NaN    NaN         NaN         NaN    NaN    NaN  
90 Minute IPA                 NaN    NaN         NaN         NaN    NaN    NaN  

[5 rows x 22140 columns]

In [9]:
# any cells that are missing data (i.e. a user didn't buy a particular product)
# we're going to set to 0
df_wide = df_wide.fillna(0)

In [15]:
# this is the key. we're going to use cosine_similarity from scikit-learn
# to compute the distance between all beers
print "calculating similarity"
dists = cosine_similarity(df_wide)

# stuff the distance matrix into a dataframe so it's easier to operate on
dists = pd.DataFrame(dists, columns=df_wide.index)

# give the indicies (equivalent to rownames in R) the name of the beer
dists.index = dists.columns

def get_similar(beers,n=None):
    """
    get_sims takes a distance matrix an beer
    and will calculate the 10 most similar products to product based on the
    distance matrix

    dists - a distance matrix
    beers - list of beers that the user likes
    """
    beers = [beer for beer in beers if beer in dists.columns]
    beers_summed = dists[beers].apply(lambda row: np.sum(row), axis=1)
    beers_summed = beers_summed.sort_values(ascending=False)
    ranked_beers = beers_summed.index[beers_summed.index.isin(beers)==False]
    ranked_beers = ranked_beers.tolist()
    if n is None:
        return ranked_beers
    else:
        return ranked_beers[:n]
    
#    p = dists[products].apply(lambda row: np.sum(row), axis=1)
#    p = p.order(ascending=False)
#    if n is None:
#        return p.index[p.index.isin(products)==False]
#    else:
#        return p.index[p.index.isin(products)==False][:n]

get_similar(["Sierra Nevada Pale Ale", "120 Minute IPA", "Coors Light"],10)

calculating similarity


['Samuel Adams Boston Lager',
 'Sierra Nevada Celebration Ale',
 '90 Minute IPA',
 'Arrogant Bastard Ale',
 'Stone IPA (India Pale Ale)',
 '60 Minute IPA',
 'HopDevil Ale',
 'Stone Ruination IPA',
 'Sierra Nevada Bigfoot Barleywine Style Ale',
 'Storm King Stout']

In [28]:
dists.to_pickle('beer.pkl') 